# Cycle Times
The time a cycle starts and its duration are two common metrics for segmenting battery datasets before analysis.
The `CycleTimes` class computes them based on the data from labels of `cycle_number` in the raw data.

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from batdata.postprocess.cycle_stats import CycleTimes
from batdata.data import BatteryDataset
import pandas as pd

## Load Example Dataset
The "single-resistor" datasets provided with battery-data-toolkit includes only a single cycle. 
We'll duplicate it to provide an example for this cycle.

In [2]:
dataset = BatteryDataset.from_batdata_hdf('../../tests/files/example-data/single-resistor-constant-charge_from-charged.hdf')
dataset.raw_data.tail()

,test_time,current,voltage,cycle_number
2396,7188.0,1.0,3.096667,0
2397,7191.0,1.0,3.097500,0
2398,7194.0,1.0,3.098333,0
2399,7197.0,1.0,3.099167,0
2400,7200.0,1.0,3.100000,0


Make a copy of the cycle, increment the `cycle_number` and advance the `test_time`.

In [3]:
cycle_two = dataset.raw_data.copy()
cycle_two['cycle_number'] += 1
cycle_two['test_time'] += cycle_two['test_time'].max() + cycle_two['test_time'].iloc[1]
cycle_two.head()

,test_time,current,voltage,cycle_number
0,7203.0,-1.0,2.899167,1
1,7206.0,-1.0,2.898333,1
2,7209.0,-1.0,2.897500,1
3,7212.0,-1.0,2.896667,1
4,7215.0,-1.0,2.895833,1


In [4]:
dataset.raw_data = pd.concat([dataset.raw_data, cycle_two], ignore_index=True)

## Compute the Cycle Time
`CycleTimes` class computes two features: 
- `cycle_start`: The time elapsed between first measurement of the cycle and the first measurement for the battery
- `cycle_duration`: The time elapsed between the start of the cycle and the beginning of the next cycle

In [5]:
computer = CycleTimes()
computer.column_names

['cycle_start', 'cycle_duration']

Compute them by calling the `compute_features` function.

In [6]:
computer.compute_features(dataset)

,cycle_number,cycle_start,cycle_duration
0,0,0.0,7203.0
1,1,7203.0,7200.0


The first cycle starts at 0 seconds and the second starts 7203 seconds later, yielding a cycle duration of 7203 seconds.

There is no third cycle in our data file, so we only report the duration of the next cycle as the time between its first and last measurement: 7200 seconds.